In [8]:
!pip install pandas
!pip install selenium


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [22]:
import shutil
def split_csv(input_file, chunk_size=5000):
    # Read the original CSV file
    df = pd.read_csv(input_file)

    # Calculate the number of chunks needed
    num_chunks = (len(df) + chunk_size - 1) // chunk_size

    # Split the dataframe into chunks
    chunks = [df[i*chunk_size:(i+1)*chunk_size] for i in range(num_chunks)]

    # Write each chunk to a separate CSV file
    for i, chunk in enumerate(chunks):
        output_file = f"data/links/link_data_{i + 1}.csv"
        chunk.to_csv(output_file, index=False)

    print(f"{num_chunks} files created.")

split_csv("data/links/link_data.csv")

4 files created.


In [9]:
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

In [10]:
df_link_data = pd.read_csv('data/link_data_1.csv')
df_link_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8002 entries, 0 to 8001
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  8002 non-null   int64 
 1   link        8002 non-null   object
dtypes: int64(1), object(1)
memory usage: 125.2+ KB


In [11]:
LIST_LINK_PRODUCT = df_link_data['link'].values.tolist()

print(len(LIST_LINK_PRODUCT))

8002


In [18]:
LIST_PRODUCT = []

# Define a function to extract property details
def extract_property_details(url):
    property_details = {
        'type_estate': '',
        'district': '',
        'price_per_sqm': '',
        'posted_date': '',
        'area': '',
        'price': '',
        'legal_document': '',
        'interior': '',
        'num_bedrooms': '',
        'num_bathrooms': '',
        'num_floors': '',
        'house_orientation': '',
        'balcony_orientation': '',
        'entrance': '',
        'frontage': ''
    }

    try:
        # Set up Selenium WebDriver
        with webdriver.Chrome() as driver:
            driver.get(url)
            
            # Get the page source
            page_source = driver.page_source

        soup = BeautifulSoup(page_source, "html.parser")

        try:
            # Find the breadcrumb element
            breadcrumb = soup.find('div', class_='re__breadcrumb')
            if breadcrumb:
                level_4_link = breadcrumb.find('a', {'level': '4'})
                if level_4_link:
                    level_4_title = level_4_link.text

                    # Use regular expressions to extract the type_estate
                    match = re.search(r"(.+?) tại", level_4_title)
                    type_estate = match.group(1) if match else ""
                    property_details['type_estate'] = type_estate

                level_3_link = breadcrumb.find('a', {'level': '3'})
                if level_3_link:
                    property_details['district'] = level_3_link.text
            else:
                print(f"Breadcrumb not found for URL: {url}")
        except AttributeError as e:
            print(f"Error extracting type_estate and district: {e}")

        # Extract price per square meter
        try:
            price_per_sqm_span = soup.find('span', class_='ext')
            if price_per_sqm_span:
                property_details['price_per_sqm'] = price_per_sqm_span.text.strip()
        except AttributeError as e:
            print(f"Error extracting price_per_sqm: {e}")

        # Extract posted date
        try:
            posted_date_span = soup.find('span', string='Ngày đăng')
            if posted_date_span:
                property_details['posted_date'] = posted_date_span.find_next_sibling('span').text.strip()
        except AttributeError as e:
            print(f"Error extracting posted_date: {e}")

        # Extract property specifications
        try:
            specs_content = soup.find_all('div', class_='re__pr-specs-content-item')
            # Mapping of titles to corresponding keys in property_details
            title_map = {
                'Diện tích': 'area',
                'Mức giá': 'price',
                'Pháp lý': 'legal_document',
                'Nội thất': 'interior',
                'Số phòng ngủ': 'num_bedrooms',
                'Số toilet': 'num_bathrooms',
                'Số tầng': 'num_floors',
                'Hướng nhà': 'house_orientation',
                'Hướng ban công': 'balcony_orientation',
                'Đường vào': 'entrance',
                'Mặt tiền': 'frontage'
            }

            # Extract and map property details
            for item in specs_content:
                title = item.find(class_='re__pr-specs-content-item-title').text.strip()
                if title in title_map:
                    property_details[title_map[title]] = item.find(class_='re__pr-specs-content-item-value').text.strip()
        except AttributeError as e:
            print(f"Error extracting property details: {e}")
        
    except Exception as e:
        print(f"Failed to extract data from {url}: {e}")

    return property_details

def scrape_property_details(urls):
    scraped_properties = []
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = {executor.submit(extract_property_details, url): url for url in urls}
        try:
            for future in as_completed(futures):
                url = futures[future]
                try:
                    result = future.result()
                    print(result)  # Print the extracted details
                    scraped_properties.append(result)
                except Exception as e:
                    print(f"Failed to extract data from {url}: {e}")
        except KeyboardInterrupt:
            for f in futures:
                f.cancel()
            raise

    return scraped_properties

LIST_PRODUCT = scrape_property_details(LIST_LINK_PRODUCT)
print(LIST_PRODUCT)

{'type_estate': 'Nhà riêng', 'district': 'Thanh Xuân', 'price_per_sqm': '~145,46 triệu/m²', 'posted_date': '08/05/2024', 'area': '55 m²', 'price': '8 tỷ', 'legal_document': '', 'interior': '', 'num_bedrooms': '', 'num_bathrooms': '', 'num_floors': '4 tầng', 'house_orientation': '', 'balcony_orientation': '', 'entrance': '', 'frontage': '4,3 m'}
{'type_estate': 'Nhà riêng', 'district': 'Long Biên', 'price_per_sqm': '~236,14 triệu/m²', 'posted_date': '15/04/2024', 'area': '83 m²', 'price': '19,6 tỷ', 'legal_document': 'Sổ đỏ.', 'interior': '', 'num_bedrooms': '', 'num_bathrooms': '', 'num_floors': '4 tầng', 'house_orientation': '', 'balcony_orientation': '', 'entrance': '8 m', 'frontage': '4,8 m'}
{'type_estate': 'Nhà riêng', 'district': 'Hoàng Mai', 'price_per_sqm': '~197,44 triệu/m²', 'posted_date': '10/05/2024', 'area': '39 m²', 'price': '7,7 tỷ', 'legal_document': 'Sổ đỏ/ Sổ hồng.', 'interior': 'Đầy đủ.', 'num_bedrooms': '3 phòng', 'num_bathrooms': '3 phòng', 'num_floors': '4 tầng', 

KeyboardInterrupt: 

In [13]:
print(len(LIST_PRODUCT))

8002


In [17]:
df_link_product = pd.DataFrame(LIST_PRODUCT)
df_link_product.to_csv('data/raw_data_1.csv', index=False)

In [15]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv("data/raw_data.csv")

# Drop the 'link' and 'describe' columns
df.drop(['link', 'describe'], axis=1, inplace=True)

# Save the modified DataFrame back to a CSV file
df.to_csv("data/raw_data.csv", index=False)

KeyError: "['link', 'describe'] not found in axis"

In [16]:
df = pd.read_csv('data/raw_data_1.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8002 entries, 0 to 8001
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           8002 non-null   int64 
 1   type_estate          7321 non-null   object
 2   district             7321 non-null   object
 3   price_per_sqm        6541 non-null   object
 4   posted_date          7314 non-null   object
 5   area                 7314 non-null   object
 6   price                7314 non-null   object
 7   legal_document       6415 non-null   object
 8   interior             4499 non-null   object
 9   num_bedrooms         5155 non-null   object
 10  num_bathrooms        4628 non-null   object
 11  num_floors           6456 non-null   object
 12  house_orientation    1545 non-null   object
 13  balcony_orientation  980 non-null    object
 14  entrance             3553 non-null   object
 15  frontage             5024 non-null   object
dtypes: int